# RAG Componets Design

## [Multi-Schemas Architecture]

### 1. How to [update] in Compile with LangGraph Framework

In [1]:
from typing import TypedDict

# 1. 사용자가 정의한 Schema (청사진)
class GraphState(TypedDict):
    one: str
    two: str
    three: str
    four: str
    five: str
    # six는 여기에 없습니다!

# 2. 노드 함수들 (사용자가 작성한 로직)
def node_a(state):
    # two를 업데이트하고, 스키마에 없는 six를 반환 시도
    return {"two": "Updated Two", "six": "I am Six"}

# ==========================================================
# [엔진 시뮬레이션] 보이지 않던 루프를 코드로 구현
# ==========================================================

# 1. 초기 상태 생성 (메모리 할당)
# 초기에는 값들이 비어있거나 None일 수 있습니다.
current_state = {
    "one": None, "two": "Initial Two", "three": None, 
    "four": None, "five": None
}

print(f"[초기 상태] {current_state}")

# 2. 실행 루프 (GraphRunner)
nodes_to_run = [node_a] # 실행할 노드 순서

for node_func in nodes_to_run:
    print(f"\n--- '{node_func.__name__}' 실행 중 ---")
    
    # (A) 함수 실행: 현재 상태를 넣어줌
    node_output = node_func(current_state)
    print(f"[노드 반환값] {node_output}") 
    # 결과: {'two': 'Updated Two', 'six': 'I am Six'}
    
    # (B) ★ 상태 업데이트 및 스키마 검증 (핵심 답변) ★
    # 엔진은 무작정 update() 하지 않고, 스키마(GraphState)에 있는 키인지 확인합니다.
    # (실제 구현은 설정에 따라 다르지만, 기본 개념은 '필터링'입니다)
    
    valid_updates = {}
    for key, value in node_output.items():
        if key in current_state:  # GraphState에 정의된 키인가?
            valid_updates[key] = value
        else:
            print(f"⚠️ [경고] '{key}'는 GraphState에 정의되지 않아 버려집니다.")
            # six는 여기서 버려짐!
            
    # (C) 유효한 값만 병합
    current_state.update(valid_updates)
    print(f"[갱신 후 상태] {current_state}")

[초기 상태] {'one': None, 'two': 'Initial Two', 'three': None, 'four': None, 'five': None}

--- 'node_a' 실행 중 ---
[노드 반환값] {'two': 'Updated Two', 'six': 'I am Six'}
⚠️ [경고] 'six'는 GraphState에 정의되지 않아 버려집니다.
[갱신 후 상태] {'one': None, 'two': 'Updated Two', 'three': None, 'four': None, 'five': None}


### 2 GlobalState & Partial Schemas

In [2]:
from typing import TypedDict, List, Optional

# 1. 전역 상태 (Global State)
class RAGGlobalState(TypedDict):
    question: str
    optimized_query: str
    retrieved_docs: List[str]
    final_answer: str

# 2. 부분 스키마 (Partial Schemas)

# 2.1. 쿼리 변환 노드의 출력 명세
class QueryOutput(TypedDict):
    optimized_query: str

# 2.2. 검색 노드의 출력 명세
class RetrievalOutput(TypedDict):
    retrieved_docs: List[str]

# 2.3. 답변 생성 노드의 출력 명세
class GenerationOutput(TypedDict):
    final_answer: str

### 3 Implementation Detail

In [3]:
from langgraph.graph import StateGraph, END

# --- [Node 1] 쿼리 최적화 (Query Rewrite) ---
# 입력: 전역 상태 전체 (혹은 필요한 필드만 명시한 InputSchema 사용 가능)
def query_rewrite_node(state: RAGGlobalState) -> QueryOutput:
    print(f"--- [1. Query Rewrite] Input: {state['question']} ---")
    
    # (LLM 로직 대체) 질문을 검색 친화적으로 변경
    new_query = f"{state['question']} (검색어 강화됨)"
    
    # 약속된 QueryOutput 스키마에 맞춰 반환 -> 전역 상태의 'optimized_query'에 병합됨
    return {"optimized_query": new_query}


# --- [Node 2] 문서 검색 (Retriever) ---
def retrieve_node(state: RAGGlobalState) -> RetrievalOutput:
    query = state['optimized_query'] # 앞 단계에서 생성된 데이터 사용
    print(f"--- [2. Retriever] Searching for: {query} ---")
    
    # (Vector DB 검색 로직 대체)
    docs = [
        f"문서 A: {query}에 대한 내용입니다.",
        f"문서 B: 관련된 또 다른 정보입니다."
    ]
    
    # 약속된 RetrievalOutput 스키마 반환 -> 전역 상태의 'retrieved_docs'에 병합됨
    return {"retrieved_docs": docs}


# --- [Node 3] 답변 생성 (Generator) ---
def generate_node(state: RAGGlobalState) -> GenerationOutput:
    q = state['question']
    docs = state['retrieved_docs']
    print(f"--- [3. Generator] Generating answer with {len(docs)} docs ---")
    
    # (LLM 생성 로직 대체)
    answer = f"질문 '{q}'에 대해, 검색된 문서({docs[0]}...)를 바탕으로 답변합니다."
    
    # 약속된 GenerationOutput 스키마 반환 -> 전역 상태의 'final_answer'에 병합됨
    return {"final_answer": answer}


# --- [Graph 조립] ---
workflow = StateGraph(RAGGlobalState)

# 노드 추가
workflow.add_node("rewrite", query_rewrite_node)
workflow.add_node("retrieve", retrieve_node)
workflow.add_node("generate", generate_node)

# 엣지 연결 (순서 정의)
workflow.set_entry_point("rewrite")
workflow.add_edge("rewrite", "retrieve")
workflow.add_edge("retrieve", "generate")
workflow.add_edge("generate", END)

# 컴파일 (엔진 생성)
app = workflow.compile()

# 1. Query Rewrite Node Design

## 1.1) imports & schemas

In [4]:
from typing import List, TypedDict, Annotated
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# 1. Graph State
class RAGState(TypedDict):
    question: str
    optimized_queries: List[str]

# 2. Pydantic Model
class RewriteResult(BaseModel):
    queries: List[str] = Field(description="검색 엔진에 최적화된, 5개 내외의 재작성된 쿼리 리스트")

/mnt/data1/work/sm-ai-v2/my-rag-server/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# 시스템 프롬프트: 작업 지시
SYSTEM_PROMPT = """
당신은 고도로 훈련된 검색 쿼리 최적화 전문가(AI Research Optimizer)입니다.
사용자의 질문을 입력받아, 벡터 데이터베이스(Vector DB) 검색에 적합하도록 다음 규칙에 따라 처리하세요.

1. **모호성 제거 (Disambiguation):** 대명사(이것, 그곳 등)가 있다면 명확한 명사로 대체하여 '독립적인 질문(Stand-alone Query)'으로 만드세요.
2. **다각도 확장 (Expansion):** 원본 질문의 의도를 유지하되, 검색 확률을 높이기 위해 서로 다른 키워드나 표현을 사용한 질문 5개를 생성하세요.
3. **언어 유지:** 질문이 한국어면 한국어로, 영어면 영어로 작성하세요.

출력은 반드시 주어진 JSON 포맷(queries 리스트)을 따르세요.
"""

rewrite_prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("human", "질문: {question}"),
])

## 1.2) The Node

In [6]:
# LLM Calling -> Parsing -> State Return
def query_rewrite_node(state: RAGState) -> dict:
    """
    사용자 질문을 받아 검색에 최적화된 쿼리 리스트를 생성하는 노드
    """
    print(f"--- [Step 1] Query Rewrite 시작: {state['question']} ---")

    # 1. Model
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    # 2. Structured Output
    structured_llm = llm.with_structured_output(RewriteResult)

    # 3. Chain
    chain = rewrite_prompt | structured_llm

    # 4. 실행
    try:
        result: RewriteResult = chain.invoke(
            {
                "question": state["question"]
            }
        )
        # 5. 결과 반환
        print(f"--- 생성된 쿼리: {result.queries} ---")
        return {"optimized_queries": result.queries}

    except Exception as e:
        print(f"Error in rewrite Node: {e}")
        return {"optimized_queries": [state["question"]]}

## 1.3) TEST

In [7]:
from dotenv import load_dotenv
load_dotenv()

# 테스트 데이터
dummy_input = {"question": "그거 환불 규정이 어떻게 돼? 아이폰 산거 말이야"}

# 노드 실행
output = query_rewrite_node(dummy_input)

# 결과 확인
print(output)

--- [Step 1] Query Rewrite 시작: 그거 환불 규정이 어떻게 돼? 아이폰 산거 말이야 ---
--- 생성된 쿼리: ['아이폰 환불 규정은 어떻게 되나요?', '아이폰 구매 후 환불 절차는 무엇인가요?', '아이폰 환불 정책에 대해 알고 싶어요.', '아이폰을 환불하려면 어떤 조건이 필요한가요?', '아이폰 환불 시 주의해야 할 사항은 무엇인가요?'] ---
{'optimized_queries': ['아이폰 환불 규정은 어떻게 되나요?', '아이폰 구매 후 환불 절차는 무엇인가요?', '아이폰 환불 정책에 대해 알고 싶어요.', '아이폰을 환불하려면 어떤 조건이 필요한가요?', '아이폰 환불 시 주의해야 할 사항은 무엇인가요?']}
